In [1]:
from wav2vec2 import Wav2Vec2Config, Wav2Vec2ForCTC
from transformers import Wav2Vec2ForCTC as HFWav2Vec2ForCTC

import tensorflow as tf
import torch
import numpy as np

def get_difference(tf_out, hf_out):
    return np.max(tf_out.numpy() - hf_out.numpy())

In [26]:
# np.random.seed(0)
seqlen = 41000
array = np.random.rand(1*seqlen).reshape(1, seqlen)
import soundfile as sf
def fetch_audio(f):
    audio, samplerate = sf.read(f)
    print({"sample_rate": samplerate})
    return audio[None, :41000]

# array = fetch_audio("../data/sample.wav")

batch = tf.convert_to_tensor(array, dtype=tf.float32)
hf_batch = torch.from_numpy(array).float()

batch, hf_batch

(<tf.Tensor: shape=(1, 41000), dtype=float32, numpy=
 array([[0.2744185 , 0.12965697, 0.3025697 , ..., 0.7224068 , 0.9908958 ,
         0.0811685 ]], dtype=float32)>,
 tensor([[0.2744, 0.1297, 0.3026,  ..., 0.7224, 0.9909, 0.0812]]))

In [3]:
tf_model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-base-960h", input_shape=batch.shape)

Loading weights locally from `wav2vec2-base-960h`
Total number of loaded variables: 212


In [4]:
hf_model = HFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
tf_out = tf_model(batch, training=False)["logits"]

In [29]:
with torch.no_grad():
    hf_out = hf_model(hf_batch)["logits"]

In [30]:
tf_out.shape, hf_out.shape

(TensorShape([1, 127, 32]), torch.Size([1, 127, 32]))

In [31]:
print("difference in logits:", get_difference(tf_out, hf_out))

difference in logits: 3.683567e-05


In [15]:
@tf.function(autograph=True, jit_compile=True)
def tf_forward(*args, **kwargs):
    return tf_model(*args, **kwargs)

In [33]:
tf_out = tf_forward(batch, training=False)["logits"]

In [34]:
print("difference in graph based model logits:", get_difference(tf_out, hf_out))

difference in graph based model logits: 3.683567e-05
